# Computing a Self-Consistent Fixed Point

This tutorial demonstrates how to compute a fixed point of the self-consistent noisy transfer operator for the logistic map.

## Mathematical Setup

We consider the logistic map $T_a(x) = a \cdot x(1-x)$ on the torus $\mathbb{T} = [0,1)$ with periodized Gaussian noise of variance $\sigma^2$.

The self-consistent operator is:
$$\mathcal{T}(f) = P_{c(f)} f$$

where $c(f) = \delta \cdot G(m(f))$ and $m(f) = \langle \cos(2\pi x), f \rangle$.

Our goal is to find a fixed point $f^*$ such that $\mathcal{T}(f^*) = f^*$.

## Step 1: Load the Package

In [ ]:
using Pkg
Pkg.activate("..")

using SelfConsistentLogisticNoise
using LinearAlgebra

## Step 2: Set Parameters

We fix:
- $a = 3.83$ (logistic parameter, in the chaotic regime)
- $\sigma = 0.02$ (noise standard deviation)
- $\delta = 0.1$ (coupling strength)
- $N = 64$ (Fourier truncation level)

In [ ]:
a = 3.83      # Logistic parameter
σ = 0.02      # Noise standard deviation
δ = 0.1       # Coupling strength
N = 64        # Fourier truncation (2N+1 = 129 modes)

## Step 3: Build the Problem

The `build_problem` function constructs:
1. The logistic map $T_a$
2. The Gaussian noise kernel $\bar{\rho}_\sigma$
3. The Fourier discretization parameters
4. The coupling (here, linear: $G(m) = m$)
5. The transfer matrix $B_a$ (computed via FFT)

In [ ]:
prob = build_problem(
    a = a,
    σ = σ,
    N = N,
    δ = δ,
    coupling_type = :linear,  # G(m) = m
    cache = true              # Cache B matrix to disk
)

In [ ]:
# Examine the problem structure
println("Map parameter a = ", prob.map.a)
println("Noise σ = ", prob.noise.σ)
println("Fourier modes: N = ", prob.disc.N, " (", 2*prob.disc.N+1, " coefficients)")
println("FFT grid size M = ", prob.disc.M)
println("Coupling δ = ", get_delta(prob.coupling))

## Step 4: Solve for the Fixed Point

We have three solver options:

### Option A: Picard Iteration (simple, robust)
$$\hat{f}^{(n+1)} = (1-\alpha) \hat{f}^{(n)} + \alpha \, \widehat{\mathcal{T}(f^{(n)})}$$

Linear convergence, but very robust.

### Option B: Newton's Method (fast, needs good initial guess)
$$(I - D\mathcal{T}(f_k)) h_k = \mathcal{T}(f_k) - f_k, \quad f_{k+1} = f_k + h_k$$

Quadratic convergence near the fixed point.

### Option C: Hybrid (recommended)
Use Picard to get close, then switch to Newton for fast final convergence.

In [ ]:
# Option A: Picard iteration
println("=== Picard Iteration ===")
@time result_picard = solve_fixed_point(
    prob;
    α = 0.3,           # Damping parameter (0 < α ≤ 1)
    tol = 1e-12,       # Convergence tolerance
    maxit = 5000,      # Maximum iterations
    init = :uniform,   # Start from uniform density
    verbose = false
)
println("Converged: ", result_picard.converged)
println("Iterations: ", result_picard.iterations)
println("Residual: ", result_picard.residual)

In [ ]:
# Option B: Pure Newton (may fail without good initial guess)
println("\n=== Newton's Method ===")
@time result_newton = solve_newton(
    prob;
    tol = 1e-12,
    maxit = 50,
    init = result_picard.fhat,  # Use Picard result as initial guess
    verbose = true
)
println("Converged: ", result_newton.converged)
println("Iterations: ", result_newton.iterations)
println("Residual: ", result_newton.residual)

In [ ]:
# Option C: Hybrid solver (recommended)
println("\n=== Hybrid Solver (Picard → Newton) ===")
@time result_hybrid = solve_hybrid(
    prob;
    picard_tol = 1e-6,    # Switch to Newton when residual < 1e-6
    newton_tol = 1e-12,   # Final tolerance
    α = 0.3,
    verbose = true
)
println("Converged: ", result_hybrid.converged)
println("Total iterations: ", result_hybrid.iterations)
println("  Picard: ", result_hybrid.params[:picard_iters])
println("  Newton: ", result_hybrid.params[:newton_iters])
println("Residual: ", result_hybrid.residual)

In [ ]:
# Use hybrid result for the rest of the tutorial
result = result_hybrid
fhat = result.fhat

println("\n=== Final Result ===")
println("Observable m(f*) = ", result.m)
println("Shift c(f*) = ", result.c)

## Step 5: Verify Normalization and Reality

A density must satisfy:
1. $\hat{f}_0 = 1$ (integral equals 1)
2. $\hat{f}_{-k} = \overline{\hat{f}_k}$ (real-valued density)

In [ ]:
# Check normalization
println("Normalization check: f̂₀ = ", fhat[idx(0, N)])

# Check conjugate symmetry (reality)
max_asymmetry = maximum(abs(fhat[idx(k, N)] - conj(fhat[idx(-k, N)])) for k in 1:N)
println("Max conjugate asymmetry: ", max_asymmetry)

## Step 6: Reconstruct the Density

Convert from Fourier coefficients to real-space density.

In [ ]:
x_grid, f_density = reconstruct_density(fhat; npts = 500)

# Verify the density is real and non-negative (approximately)
println("Density properties:")
println("  Max imaginary part: ", maximum(abs.(imag.(f_density))))
println("  Min value: ", minimum(real.(f_density)))
println("  Max value: ", maximum(real.(f_density)))
println("  Integral (trapezoidal): ", sum(f_density) * (x_grid[2] - x_grid[1]))

## Step 7: Plot the Density

Visualize the fixed-point density.

In [ ]:
using Plots

plot(x_grid, real.(f_density), 
     label="f*(x)", 
     xlabel="x", 
     ylabel="Density",
     title="Self-consistent fixed point (a=$a, σ=$σ, δ=$δ)",
     linewidth=2,
     legend=:topright)

## Step 8: Examine Fourier Coefficients

The Fourier coefficients should decay for smooth densities.

In [ ]:
println("Fourier coefficient magnitudes:")
for k in [0, 1, 2, 5, 10, 20, N]
    println("  |f̂_$k| = ", abs(fhat[idx(k, N)]))
end

In [ ]:
# Plot coefficient decay
ks = 0:N
coeffs = [abs(fhat[idx(k, N)]) for k in ks]

plot(ks, coeffs, 
     yscale=:log10, 
     marker=:circle,
     markersize=3,
     label="|f̂_k|",
     xlabel="Mode k",
     ylabel="|f̂_k| (log scale)",
     title="Fourier coefficient decay")

## Step 9: Effect of the Coupling

Compare with $\delta = 0$ (no self-consistency).

In [ ]:
prob_uncoupled = build_problem(a=a, σ=σ, N=N, δ=0.0)
result_uncoupled = solve_hybrid(prob_uncoupled; picard_tol=1e-6, newton_tol=1e-12)

println("=== Comparison: δ=0 vs δ=$δ ===")
println("δ = 0:   m = ", result_uncoupled.m, ", c = ", result_uncoupled.c)
println("δ = $δ: m = ", result.m, ", c = ", result.c)

In [ ]:
# Compare densities
_, f_uncoupled = reconstruct_density(result_uncoupled.fhat; npts = 500)

plot(x_grid, real.(f_uncoupled), label="δ=0", linewidth=2)
plot!(x_grid, real.(f_density), label="δ=$δ", linewidth=2)
xlabel!("x")
ylabel!("Density")
title!("Effect of coupling on fixed-point density")

## Step 10: Rigorous Verification (Optional)

Using BallArithmetic.jl, we can rigorously verify that a true fixed point exists near our numerical solution.

In [ ]:
# Rigorous verification
rig = verify_fixed_point(prob, result.fhat; τ=0.01, verbose=true)

if rig.verified
    println("\n✓ Rigorously verified!")
    println("  Error bound: ‖f* - f̃‖₂ ≤ ", rig.r)
    println("  Kantorovich h = ", rig.h, " ≤ 1/2")
else
    println("\n✗ Verification failed")
    println("  h = ", rig.h)
end

## Summary

We computed a self-consistent fixed point for the noisy logistic map with:
- Parameter $a = 3.83$
- Noise $\sigma = 0.02$
- Coupling $\delta = 0.1$

### Solver Comparison

| Method | Convergence | Pros | Cons |
|--------|-------------|------|------|
| Picard | Linear | Robust, simple | Slow |
| Newton | Quadratic | Fast near solution | Needs good initial guess |
| Hybrid | Both | Best of both worlds | Slightly more complex |

### Key Functions
- `build_problem()` - construct the self-consistent problem
- `solve_fixed_point()` - damped Picard iteration
- `solve_newton()` - Newton's method
- `solve_hybrid()` - Picard → Newton (recommended)
- `reconstruct_density()` - Fourier → real space
- `verify_fixed_point()` - rigorous Newton-Kantorovich verification